# Classfication and regression trees (CART)

## Algorithm description
### The CART algorithm provides the foundation for important algorithms like bagged decision trees, random forest and boosted decision trees. The output of a CART is a decision tree where each fork is a split in a predictor variable and each end node contains a prediction for the response variable. Essentially, it sequentially asks a series of if-else questions about individual features in a dataset in order to split it for classification.

## Data description and goal
### The data comes from Breast cancer Wisconsin (Diagnostic) data (https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).
### The objective is to predict whether a tumour is malignant or benign based on two features (radius_mean, concave points_mean)

# Import packages

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
import os

from six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus


# Import dataset

In [2]:
dirname = os.getcwd()

file_path = os.path.join(dirname, "Datasets", "breastcancer.csv")

breastcancer_df = pd.read_csv(file_path)

# Exploratory data analysis

In [3]:
breastcancer_df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
breastcancer_df.shape

(569, 33)

In [5]:
breastcancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [6]:
breastcancer_df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0.0
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,NaN
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,NaN
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,NaN
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


# Clean-up dataset

In [18]:
# Remove last column as it contains NA values
breastcancer_df = breastcancer_df.iloc[: , :-1]

# No need to transform any variables as CART's don't require standardised features
# Impute the missing values in the other columns

# Set-up X, y variables and test/train datasets

In [20]:
# Subset dataset to desired features
X = breastcancer_df.drop(['diagnosis'], axis =1)
y = breastcancer_df['diagnosis']
# Must replace 
#y = y.replace('M', 1).replace('B', 0)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

# Two most popular backbones for decision tree *decisions* are Gini Index and Information Entropy

## Gini index

In [23]:
# Instantiate the decision tree
dt_gini = DecisionTreeClassifier(max_depth= 8, criterion= 'gini', random_state= 1)

# Fit the decision tree to the data
dt_gini.fit(X_train, y_train)

# Predict test set labels
y_pred = dt_gini.predict(X_test)

# Calculate accuracy score
acc_dt_gini = accuracy_score(y_test, y_pred)

print(f"Accuracy score of decision tree: {acc_dt_gini}")

Accuracy score of decision tree: 0.9385964912280702


## Information Entropy


In [24]:

# Instantiate the decision tree
dt_entropy = DecisionTreeClassifier(max_depth= 8, criterion= 'entropy', random_state= 1)

# Fit the decision tree to the data
dt_entropy.fit(X_train, y_train)

# Predict test set labels
y_pred = dt_entropy.predict(X_test)

# Set new var with accuracy
acc_dt_entropy = accuracy_score(y_test, y_pred)

print(f"Accuracy score of decision tree: {acc_dt_entropy}")

Accuracy score of decision tree: 0.9473684210526315


# Visualisation of decision trees

In [13]:
feature_cols = ['radius_mean', 'concave points_mean']

dot_data = StringIO()


export_graphviz(dt_gini, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('CART_gini.png')
Image(graph.create_png())

InvocationException: GraphViz's executables not found

# Pros and Cons of CART's

## Pros

#### - Decision trees are easy to interpret and visualise - Can handle non-linear patterns